In [15]:
TIMETABLE_FILE = "C:/Users/srupe/Desktop/MTP/crew/mainLoop_aadesh.csv"

In [16]:
# === Utility Functions ===
def parse_time_to_minutes(t: str):
    """
    Converts a time string 'HH:MM' or 'HH:MM:SS' into minutes since midnight.
    Supports hours >= 24 (e.g., 24:05 → 1445 minutes, 25:07 → 1507 minutes)
    """
    try:
        t = t.strip()
        # Split HH:MM or HH:MM:SS
        parts = t.split(":")
        hours = int(parts[0])
        minutes = int(parts[1])
        return hours * 60 + minutes
    except:
        return None


def get_base_station_name(station: str):
    """Extracts the base station name (first word) in uppercase."""
    if not station:
        return None
    return station.strip().split()[0].upper()

In [17]:
class Service:
    def __init__(
        self,
        service_id, 
        rake_num,
        start_station,
        start_time,
        end_station,
        end_time,
        direction,
        service_time=0,
        same_jurisdiction=None,
        step_back_rake=None,
        step_back_location=None,
        merged_rake_num1=None,
        merged_rake_num2=None
    ):
        self.service_id = str(service_id)
        self.rake_num = rake_num
        self.start_station = start_station
        self.start_time = start_time
        self.end_station = end_station
        self.end_time = end_time
        self.direction = direction
        self.service_time = int(service_time) if service_time else 0
        self.same_jurisdiction = same_jurisdiction
        self.step_back_rake = step_back_rake
        self.step_back_location = step_back_location
        self.merged_rake_num1 = merged_rake_num1
        self.merged_rake_num2 = merged_rake_num2


def load_services(csv_file):
    df = pd.read_csv(csv_file)
    services = []
    for _, row in df.iterrows():
        s = Service(
            service_id=row.get("Service"),   
            rake_num=row.get("Rake Num"),
            start_station=get_base_station_name(row.get("Start Station")),
            start_time=parse_time_to_minutes(row.get("Start Time")),
            end_station=get_base_station_name(row.get("End Station")),
            end_time=parse_time_to_minutes(row.get("End Time")),
            direction=row.get("Direction"),
            service_time=int(row["service time"]) if "service time" in row and pd.notna(row["service time"]) else 0,
            same_jurisdiction=row["Same Jurisdiction"] if "Same Jurisdiction" in row else None,
            step_back_rake=row["Step Back Rake"] if "Step Back Rake" in row else None,
            step_back_location=row["Step Back Location"] if "Step Back Location" in row else None,
            merged_rake_num1=row["mergedRakeNum1"] if "mergedRakeNum1" in row else None,
            merged_rake_num2=row["mergedRakeNum2"] if "mergedRakeNum2" in row else None,
        )
        services.append(s)
    return services

In [18]:
# === Configuration ===
#TIMETABLE_FILE = "C:/Users/srupe/Downloads/crew/mainLoop_aadesh.csv"
MIN_RAKE_GAP_MINUTES = 30   # Minimum required gap between different rakes
ALLOWED_RAKE_CHANGE_STATIONS = {"KKDA", "PVGW"}  # Allowed rake-change stations
MAX_CONNECTION_GAP_MINUTES = 120   # Maximum allowed gap between services

BREAK_STATIONS = {"KKDA", "PVGW"}  # Breaks allowed only here
CUMULATIVE_BREAKS_DURATION = 120    # cumulative breaks should be less than CUMULATIVE_BREAKS_DURATIONS min
SHORT_BREAK = 30     # short break duration
LONG_BREAK =  50     # Long break duration
DUTY_TIME_LIMIT = 445 # Noraml duty duration
MORN_EVEN_DUTY_TIME_LIMIT = 405  # morning and evening duty time duration
MORNING_SHIFT_CUTOFF = 360     # 360 -> 6:00 AM
EVENING_SHIFT_CUTOFF = 1410     # 1410 -> 23:30 PM
CONTINUOUS_DRIVE_LIMIT = 180   # continuous driving without a break greater than 30 mins in it.
DRIVING_TIME_LIMIT =360        # Actual driving time(Mins) it have gaps< SHORT_BREAKS also counted in it


# === Jurisdiction Buckets ===
jurisdiction_dict = {
    1: {'MKPR','MKPR UP','MKPR DN','SAKP','DDSC','DDSC DN PF','DDSC SDG','DDSC SDG STABLE (DAY)','DDSC DN',
        'DDSC SDG','PVGW','PVGW UP','PVGW DN','MKPD','MKPD','SAKP 3RD','SAKP 3RD PF','MKPR DN SDG','MKPR DN PF','DDSC DN SDG'},
    2: {'MUPR DN SDG STABLE (DAY)','MUPR 4TH SDG STABLE (DAY)','MUPR 3RD SDG STABLE','SVVR','SVVR DN','MUPR',
        'MUPR DN','MUPR 4TH','MUPR 3RD SDG','KKDA DN','KKDA UP','IPE','IPE 3RD PF','IPE 3RD','VND','VND (M)',
        'MVPO','MVPO DN','NZM','NIZM','KKDA','MUPR DN SDG','MVPO DN PF','SVVR DN PF','MUPR 3RD SDG','MUPR 4TH PF',
        'MUPR 4TH SDG','MUPR DN PF','MUPR DN SDG','MUPR DN SDG'}
}


In [19]:
import pandas as pd
from tabulate import tabulate

# ---------------- PARAMETERS ----------------
MIN_RAKE_GAP_MINUTES = 30
ALLOWED_RAKE_CHANGE_STATIONS = {"KKDA", "PVGW"}
MAX_CONNECTION_GAP_MINUTES = 120

BREAK_STATIONS = {"KKDA", "PVGW"}
CUMULATIVE_BREAKS_DURATION = 120
SHORT_BREAK = 30
LONG_BREAK = 50
DUTY_TIME_LIMIT = 445
MORN_EVEN_DUTY_TIME_LIMIT = 405
MORNING_SHIFT_CUTOFF = 360
EVENING_SHIFT_CUTOFF = 1410
CONTINUOUS_DRIVE_LIMIT = 180
DRIVING_TIME_LIMIT = 360

jurisdiction_dict = {
    1: {'MKPR','MKPR UP','MKPR DN','SAKP','DDSC','DDSC DN PF','DDSC SDG','DDSC SDG STABLE (DAY)','DDSC DN',
        'DDSC SDG','PVGW','PVGW UP','PVGW DN','MKPD','MKPD','SAKP 3RD','SAKP 3RD PF','MKPR DN SDG','MKPR DN PF','DDSC DN SDG'},
    2: {'MUPR DN SDG STABLE (DAY)','MUPR 4TH SDG STABLE (DAY)','MUPR 3RD SDG STABLE','SVVR','SVVR DN','MUPR',
        'MUPR DN','MUPR 4TH','MUPR 3RD SDG','KKDA DN','KKDA UP','IPE','IPE 3RD PF','IPE 3RD','VND','VND (M)',
        'MVPO','MVPO DN','NZM','NIZM','KKDA','MUPR DN SDG','MVPO DN PF','SVVR DN PF','MUPR 3RD SDG','MUPR 4TH PF',
        'MUPR 4TH SDG','MUPR DN PF','MUPR DN SDG','MUPR DN SDG'}
}


# ---------------- HELPER FUNCTIONS ----------------
def minutes_to_hhmm(minutes):
    """Convert minutes since midnight → HH:MM format"""
    h = minutes // 60
    m = minutes % 60
    return f"{h:02d}:{m:02d}"


def get_jurisdiction(station):
    """Find jurisdiction ID for a station name"""
    for j_id, stations in jurisdiction_dict.items():
        if station in stations:
            return j_id
    return None


# ---------------- ANALYSIS FUNCTION ----------------
def analyze_duty(service_ids, all_services):
    service_dict = {s.service_id: s for s in all_services}
    duty_services = [service_dict[str(sid)] for sid in service_ids if str(sid) in service_dict]
    duty_services.sort(key=lambda s: s.start_time)

    rows = []
    total_driving_time = 0
    cumulative_breaks = 0
    rake_changes = []
    continuous_drive = 0
    max_continuous_drive = 0

    # --- Jurisdiction check ---
    first_jurisdiction = get_jurisdiction(duty_services[0].start_station)
    last_jurisdiction = get_jurisdiction(duty_services[-1].end_station)
    same_jurisdiction = first_jurisdiction == last_jurisdiction

    # --- Iterate through services ---
    for i, s in enumerate(duty_services):
        break_after = "-"
        rake_change = "-"
        gap = "-"
        
        if i < len(duty_services) - 1:
            next_s = duty_services[i + 1]
            gap = next_s.start_time - s.end_time
            
            if gap >= SHORT_BREAK:
                # OK Break detected
                break_after = f"{gap} mins OK"
                cumulative_breaks += gap
                max_continuous_drive = max(max_continuous_drive, continuous_drive)
                continuous_drive = 0
            else:
                # gap < 30 → count as continuous driving
                continuous_drive += s.service_time + gap
                total_driving_time += gap
            
            #  Check rake change
            if s.rake_num != next_s.rake_num:
                if s.end_station in ALLOWED_RAKE_CHANGE_STATIONS:
                    rake_change = f"Rake changed at {s.end_station} after {s.service_id}"
                    rake_changes.append((s.service_id, s.end_station))
                else:
                    rake_change = f" Invalid rake change at {s.end_station}"
        else:
            continuous_drive += s.service_time
        
        total_driving_time += s.service_time

        rows.append([
            s.service_id,
            minutes_to_hhmm(s.start_time),   # Start before From
            minutes_to_hhmm(s.end_time),     # End before To
            s.start_station,
            s.end_station,
            s.service_time,
            gap if gap != "-" else "-",
            break_after,
            rake_change
        ])
    
    # Update max continuous drive for last streak
    max_continuous_drive = max(max_continuous_drive, continuous_drive)

    # --- Duty & Summary ---
    duty_start = duty_services[0].start_time
    duty_end = duty_services[-1].end_time
    duty_duration = duty_end - duty_start

    summary = [
        ["Total Driving Time", f"{total_driving_time} <= {DRIVING_TIME_LIMIT}", "OK" if total_driving_time <= DRIVING_TIME_LIMIT else "Error"],
        ["Cumulative Breaks", f"{cumulative_breaks} <= {CUMULATIVE_BREAKS_DURATION}", "OK" if cumulative_breaks <= CUMULATIVE_BREAKS_DURATION else "Error"],
        ["Duty Duration", f"{duty_duration} <= {DUTY_TIME_LIMIT}", "OK" if duty_duration <= DUTY_TIME_LIMIT else "Error"],
        ["Max Continuous Drive", f"{max_continuous_drive} <= {CONTINUOUS_DRIVE_LIMIT}", "OK" if max_continuous_drive <= CONTINUOUS_DRIVE_LIMIT else "Error"],
        ["Jurisdiction Match", f"{first_jurisdiction} == {last_jurisdiction}", "OK" if same_jurisdiction else "Error"],
        ["Rake Changes", str(len(rake_changes)), ", ".join([f"{r[1]} after {r[0]}" for r in rake_changes]) or "-"]
    ]

    # --- Print Results ---
    print("\n---  Duty Timeline ---")
    print(tabulate(
        rows,
        headers=["Service_ID", "Start", "End", "From", "To", "Service Time", "Gap (mins)", "Break", "Rake Change"],
        tablefmt="pretty"
    ))

    print("\n---  Constraint Summary ---")
    print(tabulate(summary, headers=["Constraint", "Condition", "Status"], tablefmt="pretty"))


# ---------------- USAGE EXAMPLE ----------------
services = load_services(TIMETABLE_FILE)
duty_ids = [18,583,586,28,70,119,442,159]
analyze_duty(duty_ids, services)



---  Duty Timeline ---
+------------+-------+-------+------+------+--------------+------------+------------+-------------------------------+
| Service_ID | Start |  End  | From |  To  | Service Time | Gap (mins) |   Break    |          Rake Change          |
+------------+-------+-------+------+------+--------------+------------+------------+-------------------------------+
|     18     | 06:15 | 06:21 | VND  | KKDA |      6       |     0      |     -      |               -               |
|    583     | 06:21 | 06:36 | KKDA | MUPR |      15      |     4      |     -      |               -               |
|    586     | 06:40 | 06:55 | MUPR | KKDA |      15      |     0      |     -      |               -               |
|     28     | 06:55 | 08:01 | KKDA | PVGW |      66      |     45     | 45 mins OK | Rake changed at PVGW after 28 |
|     70     | 08:46 | 09:52 | PVGW | KKDA |      66      |     51     | 51 mins OK | Rake changed at KKDA after 70 |
|    119     | 10:43 | 11:50 | K

In [ ]:
import pandas as pd
from tabulate import tabulate

# ------------------ HELPER FUNCTIONS ------------------
def minutes_to_hhmm(minutes):
    h = minutes // 60
    m = minutes % 60
    return f"{h:02d}:{m:02d}"

def get_jurisdiction(station):
    for j_id, stations in jurisdiction_dict.items():
        if station in stations:
            return j_id
    return None

# ------------------ MAIN ANALYSIS ------------------
def analyze_duty(service_ids, all_services):
    service_dict = {s.service_id: s for s in all_services}
    duty_services = [service_dict[str(sid)] for sid in service_ids if str(sid) in service_dict]
    duty_services.sort(key=lambda s: s.start_time)

    rows = []
    total_driving_time = 0
    cumulative_breaks = 0
    rake_changes = []
    continuous_drive = 0
    max_continuous_drive = 0

    # Jurisdiction check
    first_jurisdiction = get_jurisdiction(duty_services[0].start_station)
    last_jurisdiction = get_jurisdiction(duty_services[-1].end_station)
    same_jurisdiction = first_jurisdiction == last_jurisdiction

    for i, s in enumerate(duty_services):
        break_after = "-"
        rake_change = "-"
        gap = "-"
        
        if i < len(duty_services) - 1:
            next_s = duty_services[i + 1]
            gap = next_s.start_time - s.end_time
            
            if gap >= SHORT_BREAK:
                break_after = f"{gap} mins OK"
                cumulative_breaks += gap
                max_continuous_drive = max(max_continuous_drive, continuous_drive)
                continuous_drive = 0
            else:
                continuous_drive += s.service_time + gap
                total_driving_time += gap
            
            if s.rake_num != next_s.rake_num:
                if s.end_station in ALLOWED_RAKE_CHANGE_STATIONS:
                    rake_change = f"Rake changed at {s.end_station} after {s.service_id}"
                    rake_changes.append((s.service_id, s.end_station))
                else:
                    rake_change = f" Invalid rake change at {s.end_station}"
        else:
            continuous_drive += s.service_time
        
        total_driving_time += s.service_time

        rows.append([
            s.service_id,
            minutes_to_hhmm(s.start_time),   # Start before From
            minutes_to_hhmm(s.end_time),     # End before To
            s.start_station,
            s.end_station,
            s.service_time,
            gap if gap != "-" else "-",
            break_after,
            rake_change
        ])
    
    
    max_continuous_drive = max(max_continuous_drive, continuous_drive)

    duty_start = duty_services[0].start_time
    duty_end = duty_services[-1].end_time
    duty_duration = duty_end - duty_start

    summary = [
        ["Total Driving Time", f"{total_driving_time} <= {DRIVING_TIME_LIMIT}", "OK" if total_driving_time <= DRIVING_TIME_LIMIT else "Error"],
        ["Cumulative Breaks", f"{cumulative_breaks} <= {CUMULATIVE_BREAKS_DURATION}", "OK" if cumulative_breaks <= CUMULATIVE_BREAKS_DURATION else "Error"],
        ["Duty Duration", f"{duty_duration} <= {DUTY_TIME_LIMIT}", "OK" if duty_duration <= DUTY_TIME_LIMIT else "Error"],
        ["Max Continuous Drive", f"{max_continuous_drive} <= {CONTINUOUS_DRIVE_LIMIT}", "OK" if max_continuous_drive <= CONTINUOUS_DRIVE_LIMIT else "Error"],
        ["Jurisdiction Match", f"{first_jurisdiction} == {last_jurisdiction}", "OK" if same_jurisdiction else "Error"],
        ["Rake Changes", str(len(rake_changes)), ", ".join([f"{r[1]} after {r[0]}" for r in rake_changes]) or "-"]
    ]

    print("\n---  Duty Timeline ---")
    print(tabulate(
        rows,
        headers=["Service", "From", "To", "Start", "End", "Service Time", "Gap (mins)", "Break", "Rake Change"],
        tablefmt="pretty"
    ))

    print("\n---  Constraint Summary ---")
    print(tabulate(summary, headers=["Constraint", "Condition", "Status"], tablefmt="pretty"))


# ------------------ RUN FOR EACH ROW IN CSV ------------------
def analyze_duty_csv(csv_file, all_services):
    with open(csv_file, "r") as f:
        lines = f.readlines()
    #df = pd.read_csv(csv_file, header=None)  # Each row is one duty
    for idx, line in enumerate(lines):
            line = line.strip()
            if not line:
                continue
            # Split by comma, convert to int
            service_ids = [int(sid.strip()) for sid in line.split(",") if sid.strip()]
            print(f"\n=================  Duty #{idx+1} =================")
            analyze_duty(service_ids, all_services)


TIMETABLE_FILE = "C:/Users/srupe/Desktop/MTP/crew/mainLoop_aadesh.csv"
services = load_services(TIMETABLE_FILE)
analyze_duty_csv("C:/Users/srupe/Desktop/MTP/crew/Solution/solution_2/solution_Network_11_10.csv", services)


=================  Duty #1 =================

---  Duty Timeline ---
+---------+------+------+-------+-------+--------------+------------+------------+--------------------------------+
| Service | From |  To  | Start |  End  | Service Time | Gap (mins) |   Break    |          Rake Change           |
+---------+------+------+-------+-------+--------------+------------+------------+--------------------------------+
|   573   | SAKP | PVGW | 18:05 | 18:08 |      3       |     0      |     -      |               -                |
|   275   | PVGW | KKDA | 18:08 | 19:14 |      66      |     0      |     -      |               -                |
|   855   | KKDA | MUPR | 19:14 | 19:29 |      15      |     2      |     -      |               -                |
|   862   | MUPR | KKDA | 19:31 | 19:47 |      16      |     0      |     -      |               -                |
|   314   | KKDA | PVGW | 19:47 | 20:53 |      66      |     51     | 51 mins OK | Rake changed at PVGW after 314 |
| 